In [ ]:
import os 

os.chdir("../..")

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch


# === 3. Set device ===
device = "mps" if torch.backends.mps.is_available() else "cpu"

def load_datasets(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['label'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'label']]
    return Dataset.from_pandas(df)

train_dataset = load_datasets(paths.english_data_dir / "train_en.tsv")
val_dataset   = load_datasets(paths.english_data_dir / "dev_en.tsv")
test_dataset  = load_datasets(paths.english_data_dir / "dev_test_en.tsv")
competition_test_dataset = load_datasets(paths.english_data_dir / "test_en_labeled.tsv")

W0615 23:35:53.990000 41040 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
model_name = "microsoft/deberta-v3-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS,
    target_modules=["query_proj", "key_proj", "value_proj", "dense"] 
)

model = get_peft_model(model, lora_config).to(device)

In [4]:
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

In [5]:
def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)
competition_test_dataset = competition_test_dataset.map(tokenize_fn, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")
competition_test_dataset = competition_test_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
competition_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [6]:
# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results/english_lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# === 10. Train ===
trainer.train()

trainer.save_model("./results/english_lora/final_checkpoint")
# === 11. Evaluate on test set ===
trainer.evaluate(eval_dataset=test_dataset)

  0%|          | 0/2080 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.6861, 'grad_norm': 1.649037480354309, 'learning_rate': 4.9759615384615386e-05, 'epoch': 0.05}
{'loss': 0.627, 'grad_norm': 3.4568028450012207, 'learning_rate': 4.9519230769230776e-05, 'epoch': 0.1}
{'loss': 0.6868, 'grad_norm': 1.6904819011688232, 'learning_rate': 4.927884615384616e-05, 'epoch': 0.14}
{'loss': 0.6399, 'grad_norm': 1.4102416038513184, 'learning_rate': 4.9038461538461536e-05, 'epoch': 0.19}
{'loss': 0.6832, 'grad_norm': 1.3337684869766235, 'learning_rate': 4.8798076923076926e-05, 'epoch': 0.24}
{'loss': 0.6224, 'grad_norm': 3.2283198833465576, 'learning_rate': 4.855769230769231e-05, 'epoch': 0.29}
{'loss': 0.5883, 'grad_norm': 1.1131833791732788, 'learning_rate': 4.8317307692307693e-05, 'epoch': 0.34}
{'loss': 0.6476, 'grad_norm': 3.1562931537628174, 'learning_rate': 4.8076923076923084e-05, 'epoch': 0.38}
{'loss': 0.6832, 'grad_norm': 1.3199255466461182, 'learning_rate': 4.783653846153847e-05, 'epoch': 0.43}
{'loss': 0.6497, 'grad_norm': 1.0305471420288086, 'l

  0%|          | 0/116 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


...

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 3267.4525, 'train_samples_per_second': 2.54, 'train_steps_per_second': 0.637, 'train_loss': 0.3846845748798492, 'epoch': 10.0}


d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/121 [00:00<?, ?it/s]

{'eval_loss': 0.8125251531600952,
 'eval_accuracy': 0.8057851239669421,
 'eval_f1_macro': 0.6946413230240549,
 'eval_precision': 0.7618357487922705,
 'eval_recall': 0.6718141472692691,
 'eval_runtime': 35.1839,
 'eval_samples_per_second': 13.756,
 'eval_steps_per_second': 3.439,
 'epoch': 10.0}

In [8]:
trainer.evaluate(eval_dataset=competition_test_dataset)

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.977461576461792,
 'eval_accuracy': 0.7466666666666667,
 'eval_f1_macro': 0.7135534448967285,
 'eval_precision': 0.7057403814301264,
 'eval_recall': 0.7378932968536251,
 'eval_runtime': 20.338,
 'eval_samples_per_second': 14.751,
 'eval_steps_per_second': 3.688,
 'epoch': 10.0}